# Tema 4 - Pamfile Alex

## Laborator 5: Jupyter lab, ipywidgets

### Exercitii:

1. Reprezentati functia $\sin(k\cdot x) \cos(x)$ si derivata ei intr-un punct ales de catre utilizator; $k$ este de asemenea un punct ales de utilizator. Pentru calculul derivatei puteti folosi formula analitica sau biblioteca [autograd](https://github.com/HIPS/autograd).
1. Incarcati fisierul de date [Abalone](http://archive.ics.uci.edu/ml/datasets/Abalone) si in functie de alegerile exprimate de un utilizator in controale de tip dropdown, afisati intr-un grafic 2D coloanele numerice alese. Numele coloanelor se gasesc in fisierul `abalone.names`.
1. Folosind un API pentru date legate de vreme (vedeti o lista de API-uri ce se pot utiliza gratuit [aici](https://rapidapi.com/blog/access-global-weather-data-with-these-weather-apis/#:~:text=What%20is%20a%20Weather%20API,weather%20forecast%20and%20historical%20information.)), creati o interfata in Jupyter notebook sau Jupyterlab, folosind ipywidgets, in care sa expuneti valori preluate din API. Definiti controale adecvate, in functie de parametrii de apel permisi de API. 

In [1]:
# # THIS IS NEEDED FOR JUPYTERLITE
# import piplite
# await piplite.install(["ipywidgets"])
# await piplite.install(["requests"])

In [2]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import math

import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
from typing import List, Tuple

print(f'matplotlib.__version__: {matplotlib.__version__}')
print(f'numpy.__version__: {np.__version__}')

matplotlib.__version__: 3.5.2
numpy.__version__: 1.21.5


1. Reprezentati functia $\sin(k\cdot x) \cos(x)$ si derivata ei intr-un punct ales de catre utilizator; $k$ este de asemenea un punct ales de utilizator. Pentru calculul derivatei puteti folosi formula analitica sau biblioteca [autograd](https://github.com/HIPS/autograd).

In [3]:
def f(x:int, k:int) -> float:
    """
    Function that represents sin(k * x) * cos(x)
    :param x: int, first parameter
    :param k: int, second parameter
    :returns: calculated result for function
    """
    return np.sin(k * x) * np.cos(x)

def f_derivative(x:int, k:int) -> float:
    """
    Function that represents derivative of f
    :param x: int, first parameter
    :param k: int, second parameter
    :returns: calculated result for derivative function
    """
    return k * np.cos(x) * np.cos(x * k) - np.sin(x) * np.sin(x *k)

def plot_f_and_f_der(min_x:float = -100, max_x:float = 100, k:float = 0):
    """
    Plots the function f and its derivative based on a range given by user
    :param min_x: int, lower bound of range for x
    :param max_x: int, upper bound of range for x
    :param k: int, input parameter
    :returns: nothing
    """
    assert min_x < max_x
    range_x:np.array = np.linspace(min_x, max_x, 100)
    values_f:np.array = f(range_x, k)
    values_d:np.array = f_derivative(range_x, k)
    plt.figure(figsize=(8, 6))
    plt.xlabel('x')
    plt.ylabel('functions')
    plt.plot(range_x, values_f, color='red')
    plt.plot(range_x, values_d, color='blue')
    plt.show()
    
interact(plot_f_and_f_der, min_x=(-100, 100.0), max_x=(-100, 100.0), k=(-50, 50.0));

interactive(children=(FloatSlider(value=-100.0, description='min_x', min=-100.0), FloatSlider(value=100.0, des…

2. Incarcati fisierul de date [Abalone](http://archive.ics.uci.edu/ml/datasets/Abalone) si in functie de alegerile exprimate de un utilizator in controale de tip dropdown, afisati intr-un grafic 2D coloanele numerice alese. Numele coloanelor se gasesc in fisierul `abalone.names`.

In [4]:
abalone_data:pd.DataFrame() = pd.read_csv('abalone.data', header=None, encoding='utf-8')
    
def plot_abalone(column1:int, column2:int) -> None:
    """
    Plots abalone dataset based on two given columns for the dataset
    :param column1: int, first column for dataset
    :param column2: int, second column for dataset
    :returns: nothing
    """
    plt.figure(figsize=(8, 8))
    plt.xlabel(list(columns.keys())[list(columns.values()).index(column1)])
    plt.ylabel(list(columns.keys())[list(columns.values()).index(column2)])
    data_m:pd.DataFrame() = abalone_data[abalone_data[0] == 'M']
    data_f:pd.DataFrame() = abalone_data[abalone_data[0] == 'F']
    data_i:pd.DataFrame() = abalone_data[abalone_data[0] == 'I']
    plt.plot(data_m.iloc[:, column1], data_m.iloc[:, column2], \
             color='blue', marker='1', linestyle='none', label = data_m.iloc[0, 0])
    plt.plot(data_f.iloc[:, column1], data_f.iloc[:, column2], \
             color='red', marker='2', linestyle='none', label = data_f.iloc[0, 0])
    plt.plot(data_i.iloc[:, column1], data_i.iloc[:, column2], \
             color='green', marker='3', linestyle='none', label = data_i.iloc[0, 0])
    plt.legend(loc='upper left')
    plt.show()


columns:dict = {"Length": 1, "Diameter": 2, "Height": 3, "Whole weight": 4, "Shucked weight": 5, "Viscera weight": 6, "Shell weight": 7, "Rings": 8}
interact(plot_abalone, column1=columns, column2=columns);

interactive(children=(Dropdown(description='column1', options={'Length': 1, 'Diameter': 2, 'Height': 3, 'Whole…

1. Folosind un API pentru date legate de vreme (vedeti o lista de API-uri ce se pot utiliza gratuit [aici](https://rapidapi.com/blog/access-global-weather-data-with-these-weather-apis/#:~:text=What%20is%20a%20Weather%20API,weather%20forecast%20and%20historical%20information.)), creati o interfata in Jupyter notebook sau Jupyterlab, folosind ipywidgets, in care sa expuneti valori preluate din API. Definiti controale adecvate, in functie de parametrii de apel permisi de API. 

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
import requests
import json

# OpenWeatherMap API keys: https://home.openweathermap.org/api_keys
api_key:str = ""

def weather_data_request(latitude:float, longitude:float) -> dict:
    """
    Makes a request for 5 day weather forecast to OpenWeatherAPI and returns resulted dictionary
    :param latitude: float, latitude for the specified location to be queried
    :param longitude: float, longitude for the specified location to be queried
    :returns: dict, result based on API call
    """
    # url:str = f"https://api.openweathermap.org/data/2.5/weather?lat={latitude}&lon={longitude}&appid={api_key}"
    url:str = f"https://api.openweathermap.org/data/2.5/forecast?lat={latitude}&lon={longitude}&appid={api_key}"
    response:str= requests.get(url, verify=False).json()
    return response

# weather_data:dict = weather_data_request("45.6427", "25.5887")

In [7]:
def get_weather_data(latitude:float, longitude:float) -> dict:
    """
    Gets the weather information for 5 days of forecast with updates every 3 hours
    :param latitude: float, latitude for the specified location to be queried
    :param longitude: float, longitude for the specified location to be queried
    :returns: dict, 5 day forecast data
    """
    weather_data:dict = weather_data_request(latitude, longitude)
    return weather_data

def weather_api(latitude:float, longitude:float, option:str) -> None:
    """
    Plots the weather data based on given option
    :param latitude: float, latitude for the specified location to be queried
    :param longitude: float, longitude for the specified location to be queried
    :param option: string, weather option (ex: temp, humidity, pressure etc)
    :returns: nothing
    """
    data:dict = get_weather_data(latitude, longitude)
    plt.figure(figsize=(8, 8))
    title:str = f'{data["city"]["name"]} ({data["city"]["country"]})'
    plt.title(title)
    plt.xlabel('day (update every 3 hours)')
    plt.ylabel(option)
    data:List[float] = [x['main'][option] for x in data['list']]
    colors:List[str] = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']
    no_days:int = 5
    intervals_in_day:int = 8
    for i in range(no_days):
        plt.plot(data[i * intervals_in_day: (i + 1) * intervals_in_day], color=colors[i], label = f'day {i + 1}')
    plt.legend(loc='upper left')
    plt.show()
    
options:List[str] = ['temp', 'feels_like', 'temp_min', 'temp_max', 'pressure', 'sea_level', 'grnd_level', 'humidity', 'temp_kf']
interact(weather_api, latitude="45.6427", longitude="25.5887", option=options);

interactive(children=(Text(value='45.6427', description='latitude'), Text(value='25.5887', description='longit…

## Laborator 5: Dashboards

1. Documentati-va despre [dashboards](https://en.wikipedia.org/wiki/Dashboard_(business)) (~ "tablouri de bord"). 
1. Documentati-va despre solutiile de dashboards disponibile in jupyter notebook, de ex:
    1. [How to create a dashboard in Python with Jupyter Notebook?](https://mljar.com/blog/dashboard-python-jupyter-notebook/)
    1. [Interactive Dashboard from Jupyter Notebook with Mercury framework](https://towardsdatascience.com/interactive-dashboard-from-jupyter-notebook-with-mercury-framework-e1269fdbe73c)
    1. [How to Create a Beautiful Python Visualization Dashboard With Panel/Hvplot](https://www.youtube.com/watch?v=uhxiXOTKzfs)
    1. [Interactive data dashboards in Jupyter notebook with ipywidgets and Bokeh](https://danielmuellerkomorowska.com/2021/08/02/interactive-data-dashboards-in-jupyter-notebook-with-ipywidgets-and-bokeh/); explicatii la [How to Build an Interactive Data Dashboard in Jupyter Notebook](https://www.youtube.com/watch?v=EEKX5l_Y3_8) etc.
1. Alegeti un set de date din repository-ul [UC Irvine Machine Learning Repository](http://archive.ics.uci.edu/ml/index.php) si realizati un dashboard pe baza lui. Prezentarea se poate face in Jupyter notebook sau ca aplicatie standalone. Includeti in arhiva:
    * fisier `requirements.txt` cu bibliotecile care trebuie instalate cu `pip`; [documentatie requirements.txt](https://learnpython.com/blog/python-requirements-file/)
    * fisier readme.txt, in limba romana sau engleza, cu pasii care trebuie urmati pentru a putea utiliza dashboard-ul; de exemplu, unul din pasi este `pip install -r requirements.txt`; adaugati alti pasi necesari. 

In [8]:
wine_data:pd.DataFrame() = pd.read_csv('wine.data', header=None, encoding='utf-8')
columns = {"Malic acid" : 1,
           "Ash": 2,
           "Alcalinity of ash": 3,
           "Magnesium": 4,
           "Total phenols": 5,
           "Flavanoids": 6,
           "Nonflavanoid phenols": 7,
           "Proanthocyanins": 8,
           "Color intensity": 9,
           "Hue": 10,
           "OD280/OD315 of diluted wines": 11,
           "Proline": 12}

def split_wine_data(wine_data:pd.DataFrame()):
    """
    Splits the wine data based on class (there are 3 classes of vendors)
    :param wine_data: pandas dataframe that represents wine dataset
    :returns: dict, 5 day forecast data
    """
    data1:pd.DataFrame() = wine_data[wine_data[0] == 1]
    data2:pd.DataFrame() = wine_data[wine_data[0] == 2]
    data3:pd.DataFrame() = wine_data[wine_data[0] == 3]
    return (data1, data2, data3)

In [9]:
def plot_wine(column1:int, column2:int) -> None:
    """
    Plots wine dataset based on two given columns for the dataset with matplotlib
    :param column1: int, first column for dataset
    :param column2: int, second column for dataset
    :returns: nothing
    """
    plt.figure(figsize=(8, 8))
    plt.xlabel(list(columns.keys())[list(columns.values()).index(column1)])
    plt.ylabel(list(columns.keys())[list(columns.values()).index(column2)])
    (data1, data2, data3) = split_wine_data(wine_data)
    plt.plot(data1.iloc[:, column1], data1.iloc[:, column2], \
             color='blue', marker='1', linestyle='none', label = data1.iloc[0, 0])
    plt.plot(data2.iloc[:, column1], data2.iloc[:, column2], \
             color='red', marker='2', linestyle='none', label = data2.iloc[0, 0])
    plt.plot(data3.iloc[:, column1], data3.iloc[:, column2], \
             color='green', marker='3', linestyle='none', label = data3.iloc[0, 0])
    plt.legend(loc='upper left')
    plt.show()
    # return fig

interact(plot_wine, column1=columns, column2=columns);

interactive(children=(Dropdown(description='column1', options={'Malic acid': 1, 'Ash': 2, 'Alcalinity of ash':…

In [16]:
import panel as pn
import numpy as np
import holoviews as hv
import hvplot.pandas

pn.extension(sizing_mode = 'stretch_width')

@pn.depends(column1=column1, column2=column2)
def wine_plot_hv(column1:int, column2:int) -> hvplot:
    """
    Plots wine dataset based on two given columns for the dataset with hvplot
    :param column1: int, first column for dataset
    :param column2: int, second column for dataset
    :returns: hvplot
    """
    # (data1, data2, data3) = split_wine_data(wine_data)
    data1:list = list(wine_data.iloc[:, column1])
    data2:list = list(wine_data.iloc[:, column2])
    return (hv.Points(data1) * hv.Points(data2)).opts(responsive=True, min_height=400)

material:pn.template = pn.template.MaterialTemplate(title='Material Template')

column1 = pn.widgets.Select(name='First', options=columns)
column2 = pn.widgets.Select(name='Second', options=columns)

material.sidebar.append(column1)
material.sidebar.append(column2)
material.main.append(pn.Card(hv.DynamicMap(wine_plot_hv)))
material.show()

Launching server at http://localhost:56080
